In [28]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import matplotlib.pyplot as plt
%matplotlib inline
import csv
from bs4 import BeautifulSoup
from datetime import datetime
import seaborn
from sklearn import linear_model
import requests

In [2]:
# Import from https://datasets.imdbws.com/
df_titles = DataFrame.from_csv("data.tsv", sep="\t")
df_titles.head(5)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tconst,,,,,,,,
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [3]:
# Import from https://datasets.imdbws.com/
df_rating = DataFrame.from_csv("data2.tsv", sep="\t")
df_rating.head(5)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,averageRating,numVotes
tconst,,
tt0000001,5.8,1520
tt0000002,6.3,185
tt0000003,6.6,1165
tt0000004,6.3,113
tt0000005,6.2,1875


In [4]:
df_merge = pd.merge(df_titles, df_rating, on='tconst')
df_merge.shape

(951197, 10)

In [5]:
df_merge.head(5)

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
tconst,,,,,,,,,,
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.8,1520
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",6.3,185
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.6,1165
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",6.3,113
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,1875


In [6]:
df_merge.drop(['isAdult'], 1, inplace=True)
df_merge.drop(['originalTitle'], 1, inplace=True)
df3 = df_merge.sort_values('numVotes', ascending=False)
df3= df3.rename(index=str, columns={"primaryTitle": "Title", "startYear": "Year", "genres": "Genres", "averageRating": "Rating IMDb" ,})
df4 = df3.head(10000)
df4.head(5)

,titleType,Title,Year,endYear,runtimeMinutes,Genres,Rating IMDb,numVotes
tconst,,,,,,,,
tt0111161,movie,The Shawshank Redemption,1994,\N,142,Drama,9.3,2111033
tt0468569,movie,The Dark Knight,2008,\N,152,"Action,Crime,Drama",9.0,2076962
tt1375666,movie,Inception,2010,\N,148,"Action,Adventure,Sci-Fi",8.8,1850525
tt0137523,movie,Fight Club,1999,\N,139,Drama,8.8,1687556
tt0110912,movie,Pulp Fiction,1994,\N,154,"Crime,Drama",8.9,1650868


In [7]:
df4.drop(['titleType'], 1, inplace=True)
df4.drop(['endYear'], 1, inplace=True)


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [8]:
df4.shape

(10000, 6)

In [9]:
# Collect from the-numbers.com by BeautifulSoup
res = requests.get("https://www.the-numbers.com/movie/budgets/all")
soup = BeautifulSoup(res.content,'lxml')
# collect links
url_list=[]
for div in soup.findAll("div", {"class":"pagination"}):
    for link in div.select("a"):
         url_list.append("https://www.the-numbers.com" + link['href'])
print(url_list)   
# loop through, collecting data, create dataframe
df_the_numbers_all = pd.DataFrame()
for link in url_list:
    res = requests.get(link)
    soup = BeautifulSoup(res.content,'lxml')
    table = soup.find_all('table')
    df8 = pd.read_html(str(table))
    df_the_numbers_all = df_the_numbers_all.append(df8)
    
df_the_numbers_all.shape
df_the_numbers_all.head(5)

['https://www.the-numbers.com/movie/budgets/all', 'https://www.the-numbers.com/movie/budgets/all/101', 'https://www.the-numbers.com/movie/budgets/all/201', 'https://www.the-numbers.com/movie/budgets/all/301', 'https://www.the-numbers.com/movie/budgets/all/401', 'https://www.the-numbers.com/movie/budgets/all/501']


,Unnamed: 0,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,1,"Dec 17, 2009",Avatar,"$425,000,000","$760,507,625","$2,789,705,275"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,795,473,000"
3,4,"Apr 22, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 13, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [10]:
df_the_numbers_all.columns = [column.strip() for column in df_the_numbers_all.columns]
df_the_numbers_all.columns
df_the_numbers_all3 = df_the_numbers_all[df_the_numbers_all.ReleaseDate != 'Unknown']

In [11]:
df_the_numbers_all3.drop(['Unnamed: 0'], 1, inplace=True)

In [12]:
df_the_numbers_all3.head(5)

,ReleaseDate,Movie,ProductionBudget,DomesticGross,WorldwideGross
0,"Dec 17, 2009",Avatar,"$425,000,000","$760,507,625","$2,789,705,275"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,795,473,000"
3,"Apr 22, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,"Dec 13, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [13]:
#Rename, sort
df_the_numbers_all2 = df_the_numbers_all3.rename(columns={"Movie": "Title", "ReleaseDate": "Year"})
df_the_numbers_all2["Year"] = pd.to_datetime(df_the_numbers_all2["Year"]).dt.strftime('%Y')
df_the_numbers_all2.head(5)
df_the_numbers_all2.sort_values('Year').tail()


,Year,Title,ProductionBudget,DomesticGross,WorldwideGross
98,2019,Wonder Park,"$100,000,000","$45,216,793","$115,115,984"
87,2020,Sonic The Hedgehog,"$90,000,000",$0,$0
14,2020,The Voyage of Doctor Dolittle,"$175,000,000",$0,$0
30,2020,Tenet,"$224,000,000",$0,$0
43,2020,Call of the Wild,"$82,000,000",$0,$0


In [14]:
df_the_numbers_all2.shape

(598, 5)

In [15]:
# Merge two dataframes
df_combo = pd.merge(df_the_numbers_all2, df4,  how='inner', on=['Title', 'Year'])
df_combo.head(5)
df_combo.sort_values('Year').head()

,Year,Title,ProductionBudget,DomesticGross,WorldwideGross,runtimeMinutes,Genres,Rating IMDb,numVotes
318,1991,Terminator 2: Judgment Day,"$100,000,000","$203,464,105","$515,419,827",137,"Action,Sci-Fi",8.5,913217
478,1992,Batman Returns,"$80,000,000","$162,833,635","$266,824,291",126,"Action,Crime,Fantasy",7.0,254007
450,1993,Last Action Hero,"$85,000,000","$50,016,394","$137,298,489",130,"Action,Adventure,Comedy",6.3,125970
324,1994,True Lies,"$100,000,000","$146,282,411","$365,300,000",141,"Action,Comedy,Thriller",7.2,220613
385,1995,Cutthroat Island,"$92,000,000","$10,017,322","$18,517,322",124,"Action,Adventure,Comedy",5.6,24212


In [16]:
#Remove ('$' and ',')
df_combo["runtimeMinutes"]  = df_combo["runtimeMinutes"].apply(lambda x: x.replace('$','').replace('\\N','0').replace(',','').replace('n/a','0'))

In [17]:
#Remove ('$' and ',')
df_combo.fillna
df_combo["WorldwideGross"]  = df_combo["WorldwideGross"].apply(lambda x: x.replace('$','').replace(',','').replace('n/a','0'))
df_combo["ProductionBudget"]  = df_combo["ProductionBudget"].apply(lambda x: x.replace('$','').replace(',','').replace('n/a','0'))
df_combo["DomesticGross"]  = df_combo["DomesticGross"].apply(lambda x: x.replace('$','').replace(',','').replace('n/a','0'))

df_combo.head(5)

,Year,Title,ProductionBudget,DomesticGross,WorldwideGross,runtimeMinutes,Genres,Rating IMDb,numVotes
0,2009,Avatar,425000000,760507625,2789705275,162,"Action,Adventure,Fantasy",7.8,1049384
1,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,136,"Action,Adventure,Fantasy",6.6,449280
2,2019,Avengers: Endgame,400000000,858373000,2795473000,181,"Action,Adventure,Sci-Fi",8.7,470989
3,2015,Avengers: Age of Ultron,330600000,459005868,1403013963,141,"Action,Adventure,Sci-Fi",7.3,670766
4,2018,Avengers: Infinity War,300000000,678815482,2048134200,149,"Action,Adventure,Sci-Fi",8.5,682591


In [18]:
df_combo.sort_values("runtimeMinutes").tail()

,Year,Title,ProductionBudget,DomesticGross,WorldwideGross,runtimeMinutes,Genres,Rating IMDb,numVotes
202,2013,The Croods,135000000,187168425,573068425,98,"Action,Adventure,Animation",7.2,177411
169,2012,Wrath of the Titans,150000000,83670083,305270083,99,"Action,Adventure,Fantasy",5.7,172843
491,2008,Bedtime Stories,80000000,110101975,221468935,99,"Comedy,Family,Fantasy",6.1,82727
509,2001,Swordfish,80000000,69772969,147080413,99,"Action,Crime,Thriller",6.5,169334
210,2018,The Nutcracker and the Four Realms,132900000,54858851,170293899,99,"Adventure,Family,Fantasy",5.5,19837


In [19]:
# Create 19 separete rows with Genres, transform booleans into int
df_combo['Action'] = ['Action' in x for x in df_combo['Genres']]
df_combo['Action']=df_combo['Action'].astype('int')
df_combo['Adventure'] = ['Adventure' in x for x in df_combo['Genres']]
df_combo['Adventure']=df_combo['Adventure'].astype('int')
df_combo['Biography'] = ['Biography' in x for x in df_combo['Genres']]
df_combo['Biography']=df_combo['Biography'].astype('int')
df_combo['Comedy'] = ['Comedy' in x for x in df_combo['Genres']]
df_combo['Comedy']=df_combo['Comedy'].astype('int')
df_combo['Crime'] = ['Crime' in x for x in df_combo['Genres']]
df_combo['Crime']=df_combo['Crime'].astype('int')
df_combo['Drama'] = ['Drama' in x for x in df_combo['Genres']]
df_combo['Drama']=df_combo['Drama'].astype('int')
df_combo['Family'] = ['Family' in x for x in df_combo['Genres']]
df_combo['Family']=df_combo['Family'].astype('int')
df_combo['History'] = ['History' in x for x in df_combo['Genres']]
df_combo['History']=df_combo['History'].astype('int')
df_combo['Horror'] = ['Horror' in x for x in df_combo['Genres']]
df_combo['Horror']=df_combo['Horror'].astype('int')
df_combo['Music'] = ['Music' in x for x in df_combo['Genres']]
df_combo['Music']=df_combo['Music'].astype('int')
df_combo['Mystery'] = ['Mystery' in x for x in df_combo['Genres']]
df_combo['Mystery']=df_combo['Mystery'].astype('int')
df_combo['Romance'] = ['Romance' in x for x in df_combo['Genres']]
df_combo['Romance']=df_combo['Romance'].astype('int')
df_combo['Sport'] = ['Sport' in x for x in df_combo['Genres']]
df_combo['Sport']=df_combo['Sport'].astype('int')
df_combo['Thriller'] = ['Thriller' in x for x in df_combo['Genres']]
df_combo['Thriller']=df_combo['Thriller'].astype('int')
df_combo['War'] = ['War' in x for x in df_combo['Genres']]
df_combo['War']=df_combo['War'].astype('int')
df_combo['Western'] = ['Western' in x for x in df_combo['Genres']]
df_combo['Western']=df_combo['Western'].astype('int')
df_combo['Fantasy'] = ['Fantasy' in x for x in df_combo['Genres']]
df_combo['Fantasy']=df_combo['Fantasy'].astype('int')
df_combo['Animation'] = ['Animation' in x for x in df_combo['Genres']]
df_combo['Animation']=df_combo['Animation'].astype('int')
df_combo['Sci-Fi'] = ['Sci-Fi' in x for x in df_combo['Genres']]
df_combo['Sci-Fi']=df_combo['Sci-Fi'].astype('int')
df_combo.drop(['Genres'], 1, inplace=True)
df_combo.shape

(521, 27)

In [20]:
df_combo


,Year,Title,ProductionBudget,DomesticGross,WorldwideGross,runtimeMinutes,Rating IMDb,numVotes,Action,Adventure,...,Music,Mystery,Romance,Sport,Thriller,War,Western,Fantasy,Animation,Sci-Fi
0,2009,Avatar,425000000,760507625,2789705275,162,7.8,1049384,1,1,...,0,0,0,0,0,0,0,1,0,0
1,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,136,6.6,449280,1,1,...,0,0,0,0,0,0,0,1,0,0
2,2019,Avengers: Endgame,400000000,858373000,2795473000,181,8.7,470989,1,1,...,0,0,0,0,0,0,0,0,0,1
3,2015,Avengers: Age of Ultron,330600000,459005868,1403013963,141,7.3,670766,1,1,...,0,0,0,0,0,0,0,0,0,1
4,2018,Avengers: Infinity War,300000000,678815482,2048134200,149,8.5,682591,1,1,...,0,0,0,0,0,0,0,0,0,1
5,2017,Justice League,300000000,229024295,655945209,120,6.5,331909,1,1,...,0,0,0,0,0,0,0,1,0,0
6,2015,Spectre,300000000,200074175,879620923,148,6.8,353803,1,1,...,0,0,0,0,1,0,0,0,0,0
7,2012,The Dark Knight Rises,275000000,448139099,1084439099,164,8.4,1393518,1,0,...,0,0,0,0,1,0,0,0,0,0
8,2018,Solo: A Star Wars Story,275000000,213767512,393151347,135,7.0,229546,1,1,...,0,0,0,0,0,0,0,1,0,0
9,2013,The Lone Ranger,275000000,89302115,260002115,150,6.4,209489,1,1,...,0,0,0,0,0,0,1,0,0,0


In [21]:
# Change dtypes
df_combo['ProductionBudget']=df_combo['ProductionBudget'].astype('int')
df_combo['WorldwideGross']=df_combo['WorldwideGross'].astype('int')
df_combo['DomesticGross']=df_combo['DomesticGross'].astype('int')
df_combo['runtimeMinutes']=df_combo['runtimeMinutes'].astype('int')
df_combo['Year']=df_combo['Year'].astype('int')
df_combo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 521 entries, 0 to 520
Data columns (total 27 columns):
Year                521 non-null int64
Title               521 non-null object
ProductionBudget    521 non-null int64
DomesticGross       521 non-null int64
WorldwideGross      521 non-null int64
runtimeMinutes      521 non-null int64
Rating IMDb         521 non-null float64
numVotes            521 non-null int64
Action              521 non-null int64
Adventure           521 non-null int64
Biography           521 non-null int64
Comedy              521 non-null int64
Crime               521 non-null int64
Drama               521 non-null int64
Family              521 non-null int64
History             521 non-null int64
Horror              521 non-null int64
Music               521 non-null int64
Mystery             521 non-null int64
Romance             521 non-null int64
Sport               521 non-null int64
Thriller            521 non-null int64
War                 521 non-null i

In [22]:
# Export
df_combo = df_combo.rename(columns={"Rating IMDb ": "Rating_IMDb ", "numVotes ": "Popularity"})
df_combo.to_csv("df_combo_genres.csv")
df_combo

,Year,Title,ProductionBudget,DomesticGross,WorldwideGross,runtimeMinutes,Rating IMDb,numVotes,Action,Adventure,...,Music,Mystery,Romance,Sport,Thriller,War,Western,Fantasy,Animation,Sci-Fi
0,2009,Avatar,425000000,760507625,2789705275,162,7.8,1049384,1,1,...,0,0,0,0,0,0,0,1,0,0
1,2011,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,136,6.6,449280,1,1,...,0,0,0,0,0,0,0,1,0,0
2,2019,Avengers: Endgame,400000000,858373000,2795473000,181,8.7,470989,1,1,...,0,0,0,0,0,0,0,0,0,1
3,2015,Avengers: Age of Ultron,330600000,459005868,1403013963,141,7.3,670766,1,1,...,0,0,0,0,0,0,0,0,0,1
4,2018,Avengers: Infinity War,300000000,678815482,2048134200,149,8.5,682591,1,1,...,0,0,0,0,0,0,0,0,0,1
5,2017,Justice League,300000000,229024295,655945209,120,6.5,331909,1,1,...,0,0,0,0,0,0,0,1,0,0
6,2015,Spectre,300000000,200074175,879620923,148,6.8,353803,1,1,...,0,0,0,0,1,0,0,0,0,0
7,2012,The Dark Knight Rises,275000000,448139099,1084439099,164,8.4,1393518,1,0,...,0,0,0,0,1,0,0,0,0,0
8,2018,Solo: A Star Wars Story,275000000,213767512,393151347,135,7.0,229546,1,1,...,0,0,0,0,0,0,0,1,0,0
9,2013,The Lone Ranger,275000000,89302115,260002115,150,6.4,209489,1,1,...,0,0,0,0,0,0,1,0,0,0
